In [46]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
from datetime import timedelta
from dateutil import relativedelta
import numpy as np
from IPython.display import display

In [47]:
timestamp = dt.now()
#timestamp = dt(2023, 11, 29, 19, 53, 49)
# source
user_credential = r'/Users/dinhhoang.nguyen.CONCENTRIX/OneDrive - Concentrix Corporation\WFM-internal'
# import master roster
masterroster = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\User\CNX Global Master Roster.xlsx'), sheet_name='Sheet1')
tl = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\TL\TL List.xlsx'), sheet_name='Sheet1')
# Import extention
extension = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\Extension\Extension.xlsx'), sheet_name='Sheet1')

In [48]:
# import the workplan
workplan_wst = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\workplan_wst.json'))

In [49]:
# A table to look up Shift time only will be added later into main data frame
shift_time = workplan_wst[['EID', 'Date', 'date_starttime', 'date_endtime', 'date_duration', 'LOB']].drop_duplicates()
shift_time['Date'] = pd.to_datetime(shift_time['Date'], format='mixed').dt.date
# Remove columns in workplan_wst to make it more concise
workplan_wst = workplan_wst.drop(columns={'date_starttime', 'date_endtime', 'date_duration', 'LOB'})

In [50]:
# A table to look up total duration need to be delivered in each aux
activity_time = workplan_wst.groupby(['EID', 'Date', 'Schedule Act'], as_index=False)['Length'].sum()
activity_time['Date'] = activity_time['Date'].dt.date
activity_time = activity_time.rename(columns={'Length': 'Total Length'})

In [51]:
# from workplan_wst, we extract current workplan of all agent on the floor
workplan_now = workplan_wst.loc[(pd.to_datetime(workplan_wst['act_starttime'], format='mixed')<=timestamp) & (pd.to_datetime(workplan_wst['act_endtime'], format='mixed')>=timestamp)]

In [52]:

# import the realtime view
rt_view = pd.read_excel(os.path.join(r'C:', user_credential, r'Data\Realtime view\00_RTA_View-Agent Team Real Time.xlsx'), sheet_name='00_RTA_View-Agent Team Real Tim')
rt_view = pd.merge(rt_view, extension[['Extension Number', 'Emp ID']], left_on='Extension', right_on='Extension Number', how='left') # add Employee ID to the table
rt_view = rt_view.drop(columns={'Extension Number'}) #to remove abundant col
rt_view = pd.merge(rt_view, masterroster[['CUIC Name', 'Employee_ID']], left_on='Agent', right_on='CUIC Name', how='left') #get agent's ID from master roster
rt_view['Emp ID'] = rt_view.apply(lambda x: x['Employee_ID'] if pd.isnull(x['Extension']) or pd.isnull(x['Emp ID']) else x['Emp ID'], axis=1) #replace ID from extension (which we ref from Extension) with ID from master roster (which we ref from CUIC name))
rt_view = rt_view.drop(columns={'Employee_ID', 'Agent Team', 'Extension', 'CUIC Name', 'Agent'}) #remove this col since this no longer necessary
rt_view = rt_view.loc[pd.notnull(rt_view['Emp ID'])]
rt_view = rt_view.rename(columns={'DateTime': 'actual_starttime'})

In [53]:
main = pd.merge(workplan_now, rt_view, left_on='EID', right_on='Emp ID', how='outer')
# main['actual_starttime'] = main.apply(lambda x: '' if pd.isnull(x['Duration']) else timestamp - timedelta(days = x['Duration']), axis=1)
main['time_stamp'] = timestamp #add a timestamp column

main['Employee_ID'] = main.apply(lambda x: x['EID'] if pd.notnull(x['EID']) else x['Emp ID'], axis=1) #since null values always exist in EID and Emp ID, now we need to create a column we null does not exists
main = main.drop(columns={'EID', 'Emp ID'}) #then we remove the 2 original columns

main['Date'] = main.apply(lambda x: x['Date'] if pd.notnull(x['Date']) else x['time_stamp'], axis=1)
main['Date'] = main['Date'].dt.date
# add additional info of agents

main = pd.merge(main, tl[['Emp ID', 'Supervisor (no need fill)']], left_on = 'Employee_ID', right_on = 'Emp ID', how='left') # add supervisor
main = main.drop(columns={'Emp ID'})
main = main.rename(columns={'Supervisor (no need fill)': 'Supervisor'})
main = pd.merge(main, masterroster[['Employee_ID', 'Wave #', 'TED Name', 'CUIC Name','Full name']], on = 'Employee_ID', how='left')

# now we add back information of the shift
main = pd.merge(main, shift_time, left_on=['Employee_ID', 'Date'], right_on=['EID', 'Date'], how='left')
main = main.drop(columns={'EID'})

main['time_stamp'] = pd.to_datetime(main['time_stamp'], format='mixed')
main['time_stamp'] = main['time_stamp'].dt.strftime('%Y-%m-%d %I:%M:%S %p')



In [54]:
# add total duration of activity
main = pd.merge(main, activity_time, left_on = ['Employee_ID', 'Date', 'Schedule Act'], right_on = ['EID', 'Date', 'Schedule Act'], how='left')
main = main.drop(columns={'EID'})

In [55]:
# Export data
timestamp = timestamp.strftime("%Y%m%d-%H%M%S")
main.to_csv(os.path.join(r'C:', user_credential, r'Data\realtime_capture', f'capture_{timestamp}.csv'), index=False)